In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import HeteroData, DataLoader
from torch_geometric.nn import RGCNConv
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero

In [8]:
nodes = pd.read_csv('test_nodes.csv')
playerFrameData = pd.read_csv('test_playerFrameData.csv')
edges = pd.read_csv('test_edges.csv')
playerEdges = pd.read_csv('test_playerEdges.csv').sort_values(by=['roundNum','floorSec']).iloc[0:10]
graph_data = pd.read_csv('test_graph_data.csv').T.rename(columns={0: 'roundNum', 1: 'floorSec', 2: 'team1AliveNum', 3: 'team2AliveNum', 4: 'CTwinsRound'})

In [9]:
nodes['x'] = nodes['x'].astype('float32')
nodes['y'] = nodes['y'].astype('float32')
playerFrameData = playerFrameData.astype('float32')

In [10]:
# Create a sample heterogeneous graph with node, edge, and multiple graph-level features
node_features = {
    'map': torch.tensor(nodes[['x','y']].values),
    'player': torch.tensor(playerFrameData.values)  # Different feature size for 'node_type2'
}

edge_index = {
    ('map', 'connected_to', 'map'): torch.tensor(edges.values),
    ('player', 'closest_to', 'map'): torch.tensor(playerEdges.loc[ (playerEdges['roundNum'] == 1) & (playerEdges['floorSec'] == 0)][['playerId','closestId']].values)
}

# Define multiple graph-level features
graph_level_features = {
    'roundNum': torch.tensor(graph_data['roundNum'].iloc[0].astype('float32')),
    'floorSec': torch.tensor(graph_data['floorSec'].iloc[0].astype('float32')),
    'team1AliveNum': torch.tensor(graph_data['team1AliveNum'].iloc[0].astype('float32')),
    'team2AliveNum': torch.tensor(graph_data['team2AliveNum'].iloc[0].astype('float32')),
}

# Convert the data to PyTorch Geometric HeteroData format
data = HeteroData(
    x=node_features,
    edge_index=edge_index,
    y=torch.tensor(graph_data['CTwinsRound'].iloc[0]),
    **graph_level_features  # Include multiple graph-level features
)

In [11]:
#data['x']['map'].shape
data['x']['player'].shape

torch.Size([10, 44])

In [12]:
data['edge_index'][('player', 'closest_to', 'map')].shape
#data['edge_index'][('map', 'connected_to', 'map')].T.shape

torch.Size([10, 2])

In [13]:
from torch_geometric.nn import HeteroConv, Linear, SAGEConv, GCNConv


class HeteroGNN(torch.nn.Module):

    def __init__(self, node_input_dims, graph_feature_dims, hidden_dim):
        super(HeteroGNN, self).__init__()
        self.conv1_type1 = GCNConv(node_input_dims['map'], hidden_dim)
        self.conv1_type2 = GCNConv(node_input_dims['player'], hidden_dim)
        self.conv2_type1 = GCNConv(hidden_dim, hidden_dim)
        self.fc = Linear(hidden_dim + graph_feature_dims, 1)  # Output dimension is 1 for binary classification

    def forward(self, x, edge_index, graph_level_features):
        print(edge_index[('player', 'closest_to', 'map')].shape)
        x_type1 = self.conv1_type1(x['map'], edge_index[('map', 'connected_to', 'map')].T)
        print("firstConvDone")
        x_type2 = self.conv1_type2(x['player'], edge_index[('player', 'closest_to', 'map')])
        print("secondConvDone")

        x_type1 = torch.relu(x_type1)
        x_type2 = torch.relu(x_type2)

        x_type1 = self.conv2_type1(x_type1, edge_index[('map', 'connected_to', 'map')])

        # Aggregating information from nodes and edges
        graph_level_representation = torch.mean(x_type1, dim=0)  # You can use different aggregation methods
        graph_level_representation = torch.cat([
            graph_level_representation,
            graph_level_features['roundNum'],
            graph_level_features['floorSec'],
            graph_level_features['team1AliveNum'],
            graph_level_features['team2AliveNum'],
        ],dim=-1)

        graph_level_prediction = self.fc(graph_level_representation)
        return graph_level_prediction


In [14]:
from torch.nn import BCEWithLogitsLoss

# Instantiate the model
node_input_dims = {'map': 2, 'player': 44}
graph_feature_dims = 4  # Total dimensions of graph-level features
model = HeteroGNN(node_input_dims=node_input_dims, graph_feature_dims=graph_feature_dims, hidden_dim=64)

# Define binary cross-entropy loss and optimizer
criterion = BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(2):
    optimizer.zero_grad()
    graph_level_prediction = model(data['x'], data['edge_index'], data.y)
    loss = criterion(graph_level_prediction, data.y.view(-1, 1))  # Reshape y for BCE loss
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


torch.Size([10, 2])
firstConvDone


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 10 but got size 2 for tensor number 1 in the list.

In [15]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['player'].train_mask
    loss = F.cross_entropy(out[mask], data['player'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)